In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_squared_error
import joblib
import warnings

warnings.filterwarnings('ignore')

# ============================================================================== 
# --- 1. CONFIGURATION
# ==============================================================================

# Updated to your uploaded files in /mnt/data
CLAIM_FILE_PATH = '/mnt/data/uk_pmi_claims_200k.csv'
MEMBER_FILE_PATH = '/mnt/data/uk_pmi_membership_120k.csv'

# Claim Columns
CLAIM_AMOUNT_COL = 'Claim Amount'
CLAIM_DATE_COL = 'Incurred Date'
CLAIM_MEMBER_ID_COL = 'Unique Member Reference'
CLAIM_TYPE_COL = 'Claim Type'
CLAIM_LOS_COL = 'Calculated Length of Service'
CONDITION_CATEGORY_COL = 'Condition Category'

# Member Columns
MEMBER_ID_COL = 'Unique Member Reference'
MEMBER_DOB_COL = 'Year of Birth'
MEMBER_JOIN_DATE_COL = 'Original Date of Joining'
MEMBER_GENDER_COL = 'Gender'
MEMBER_POSTCODE_COL = 'Short Post Code of Member'
MEMBER_SCHEME_COL = 'Scheme Category/ Section Name'
MEMBER_STATUS_COL = 'Status of Member'

START_YEAR = 2019
END_YEAR = 2025
VALIDATION_YEAR = 2025

print("Configuration loaded.")


# ============================================================================== 
# --- 2. LOAD AND CLEAN DATA
# ==============================================================================
def load_data(claim_path, member_path):
    try:
        try:
            df_claims = pd.read_csv(claim_path)
            df_members = pd.read_csv(member_path)
        except UnicodeDecodeError:
            df_claims = pd.read_csv(claim_path, encoding='latin1')
            df_members = pd.read_csv(member_path, encoding='latin1')

        print(f"Successfully loaded {claim_path} and {member_path}")

        claim_cols_to_keep = [
            CLAIM_MEMBER_ID_COL, CLAIM_DATE_COL, CLAIM_AMOUNT_COL,
            CLAIM_TYPE_COL, CLAIM_LOS_COL, CONDITION_CATEGORY_COL
        ]
        df_claims = df_claims[claim_cols_to_keep].copy()

        df_claims[CLAIM_DATE_COL] = pd.to_datetime(df_claims[CLAIM_DATE_COL], errors='coerce')
        df_claims['claim_year'] = df_claims[CLAIM_DATE_COL].dt.year
        df_claims[CLAIM_AMOUNT_COL] = pd.to_numeric(df_claims[CLAIM_AMOUNT_COL], errors='coerce')
        df_claims[CLAIM_LOS_COL] = pd.to_numeric(df_claims[CLAIM_LOS_COL], errors='coerce')
        df_claims = df_claims.dropna(subset=[CLAIM_DATE_COL, CLAIM_MEMBER_ID_COL, CLAIM_AMOUNT_COL])
        df_claims = df_claims[df_claims['claim_year'].between(START_YEAR, END_YEAR)]

        member_cols_to_keep = [
            MEMBER_ID_COL, MEMBER_DOB_COL, MEMBER_JOIN_DATE_COL,
            MEMBER_GENDER_COL, MEMBER_POSTCODE_COL, MEMBER_SCHEME_COL, MEMBER_STATUS_COL
        ]
        df_members = df_members[member_cols_to_keep].copy()
        df_members[MEMBER_JOIN_DATE_COL] = pd.to_datetime(df_members[MEMBER_JOIN_DATE_COL], errors='coerce')
        df_members[MEMBER_ID_COL] = df_members[MEMBER_ID_COL].astype(str)
        df_claims[CLAIM_MEMBER_ID_COL] = df_claims[CLAIM_MEMBER_ID_COL].astype(str)
        df_members = df_members.drop_duplicates(subset=[MEMBER_ID_COL])

        print(f"Data cleaning complete. Claims: {len(df_claims)}, Members: {len(df_members)}")
        return df_claims, df_members

    except FileNotFoundError:
        print("Error: One of the files was not found.")
        return None, None
    except Exception as e:
        print(f"Error during data loading: {e}")
        return None, None


# ============================================================================== 
# --- 3. CREATE MEMBER-YEAR SCAFFOLD
# ==============================================================================
def create_scaffold(df_claims, df_members, start_year, end_year):
    print("Creating member-year scaffold...")

    def had_condition(series, keyword):
        return (series.str.contains(keyword, case=False, na=False)).any()

    aggregation_rules = {
        'total_claim_amount': (CLAIM_AMOUNT_COL, 'sum'),
        'total_claim_count': (CLAIM_AMOUNT_COL, 'count'),
        'total_los': (CLAIM_LOS_COL, 'sum'),
        'count_inpatient': (CLAIM_TYPE_COL, lambda x: (x == 'Inpatient').any()),
        'unique_conditions_count': (CONDITION_CATEGORY_COL, 'nunique'),
        'had_cardio': (CONDITION_CATEGORY_COL, lambda s: had_condition(s, 'Cardio')),
        'had_musculo': (CONDITION_CATEGORY_COL, lambda s: had_condition(s, 'Musculo')),
        'had_gastro': (CONDITION_CATEGORY_COL, lambda s: had_condition(s, 'Gastro')),
        'had_chemo': (CONDITION_CATEGORY_COL, lambda s: had_condition(s, 'Chemo')),
    }

    # perform aggregation
    df_agg = df_claims.groupby([CLAIM_MEMBER_ID_COL, 'claim_year']).agg(**aggregation_rules).reset_index()
    df_agg = df_agg.rename(columns={'claim_year': 'year', CLAIM_MEMBER_ID_COL: MEMBER_ID_COL})

    # scaffold
    all_members = df_members[MEMBER_ID_COL].unique()
    all_years = list(range(start_year, end_year + 1))
    scaffold_index = pd.MultiIndex.from_product([all_members, all_years], names=[MEMBER_ID_COL, 'year'])
    df_scaffold = pd.DataFrame(index=scaffold_index).reset_index()

    df_master = pd.merge(df_scaffold, df_agg, on=[MEMBER_ID_COL, 'year'], how='left')

    # fill zeros for numeric-like columns
    cols_to_fill = list(aggregation_rules.keys())
    df_master[cols_to_fill] = df_master[cols_to_fill].fillna(0)

    # Ensure boolean-like aggregated cols are numeric 0/1 (int)
    bool_like_cols = ['count_inpatient', 'had_cardio', 'had_musculo', 'had_gastro', 'had_chemo']
    for c in bool_like_cols:
        if c in df_master.columns:
            df_master[c] = df_master[c].replace({True: 1, False: 0}).fillna(0).astype(int)

    # merge membership static fields
    df_master = pd.merge(df_master, df_members, on=MEMBER_ID_COL, how='left')
    df_master = df_master.dropna(subset=[MEMBER_DOB_COL, MEMBER_JOIN_DATE_COL])

    print("Scaffold creation complete.")
    return df_master


# ============================================================================== 
# --- 4. FEATURE ENGINEERING (FIXED ROLLING + SAFE CASTS)
# ==============================================================================
def engineer_features(df_master):
    print("Engineering features...")
    df_master = df_master.sort_values(by=[MEMBER_ID_COL, 'year']).reset_index(drop=True)

    g = df_master.groupby(MEMBER_ID_COL)

    def safe_rolling(series, window, func):
        # Use groupby on member id, rolling, aggregate, shift, reset index to align
        return (
            series.groupby(df_master[MEMBER_ID_COL])
            .rolling(window, min_periods=1)
            .agg(func)
            .shift(1)
            .reset_index(level=0, drop=True)
        )

    # helper
    df_master['is_claim_free_year'] = (df_master['total_claim_count'] == 0).astype(int)

    # demographics
    df_master['age'] = df_master['year'] - pd.to_numeric(df_master[MEMBER_DOB_COL], errors='coerce')
    df_master['tenure'] = df_master['year'] - df_master[MEMBER_JOIN_DATE_COL].dt.year
    df_master['tenure'] = df_master['tenure'].clip(lower=0)

    # Lag features (ensure numeric and fill NaNs)
    df_master['cost_lag1'] = g['total_claim_amount'].shift(1).fillna(0).astype(float)
    df_master['cost_lag2'] = g['total_claim_amount'].shift(2).fillna(0).astype(float)
    df_master['claim_count_lag1'] = g['total_claim_count'].shift(1).fillna(0).astype(float)

    # inpatient/count and condition lags: ensure they become numeric 0/1 or numeric
    df_master['inpatient_count_lag1'] = g['count_inpatient'].shift(1).fillna(0).astype(int)

    condition_lags = ['unique_conditions_count', 'had_cardio', 'had_musculo', 'had_gastro', 'had_chemo']
    for col in condition_lags:
        if col in df_master.columns:
            if col == 'unique_conditions_count':
                df_master[f'{col}_lag1'] = g[col].shift(1).fillna(0).astype(float)
            else:
                df_master[f'{col}_lag1'] = g[col].shift(1).fillna(0).astype(int)

    # Rolling 3-year features (safe)
    df_master['cost_avg_3yr'] = safe_rolling(df_master['total_claim_amount'], 3, 'mean').fillna(0).astype(float)
    df_master['cost_max_3yr'] = safe_rolling(df_master['total_claim_amount'], 3, 'max').fillna(0).astype(float)
    df_master['cost_std_3yr'] = safe_rolling(df_master['total_claim_amount'], 3, 'std').fillna(0).astype(float)
    df_master['claim_free_years_3yr'] = safe_rolling(df_master['is_claim_free_year'], 3, 'sum').fillna(0).astype(int)

    # Rolling 5-year features
    df_master['cost_avg_5yr'] = safe_rolling(df_master['total_claim_amount'], 5, 'mean').fillna(0).astype(float)
    df_master['cost_max_5yr'] = safe_rolling(df_master['total_claim_amount'], 5, 'max').fillna(0).astype(float)

    # Target
    df_master['target_cost_next_year'] = g['total_claim_amount'].shift(-1)

    # Prepare model dataset
    df_model_data = df_master.dropna(subset=['target_cost_next_year']).copy()

    # Make sure required lag features exist and are numeric; drop rows where critical lags are missing
    required_for_model = ['cost_lag1', 'cost_avg_3yr']
    missing_req = [c for c in required_for_model if c not in df_model_data.columns]
    if missing_req:
        raise ValueError(f"Missing required features after engineering: {missing_req}")

    df_model_data = df_model_data.dropna(subset=required_for_model)

    # Final fill for any remaining numeric NaNs
    numeric_cols = df_model_data.select_dtypes(include=['number']).columns.tolist()
    df_model_data[numeric_cols] = df_model_data[numeric_cols].fillna(0)

    # Ensure boolean-like columns are numeric
    bool_like_lag_cols = ['inpatient_count_lag1', 'had_cardio_lag1', 'had_musculo_lag1', 'had_gastro_lag1', 'had_chemo_lag1']
    for c in bool_like_lag_cols:
        if c in df_model_data.columns:
            df_model_data[c] = pd.to_numeric(df_model_data[c], errors='coerce').fillna(0).astype(int)

    # Drop helper column from final
    df_model_data = df_model_data.drop(columns=['is_claim_free_year'], errors='ignore')
    df_master = df_master.drop(columns=['is_claim_free_year'], errors='ignore')

    print(f"Feature engineering complete. Model-ready data has {len(df_model_data)} rows.")
    return df_model_data, df_master


# ============================================================================== 
# --- 5. TRAIN & VALIDATE MODEL
# ==============================================================================
def train_and_validate(df_model_data, validation_year):
    print(f"\n--- Phase 1: Training & Validation on {validation_year} ---")

    target = 'target_cost_next_year'

    static_features = [
        MEMBER_GENDER_COL, MEMBER_POSTCODE_COL,
        MEMBER_SCHEME_COL, MEMBER_STATUS_COL
    ]

    numeric_features = [
        'age', 'tenure',
        'cost_lag1', 'cost_lag2', 'claim_count_lag1', 'inpatient_count_lag1',
        'cost_avg_3yr', 'cost_max_3yr', 'cost_std_3yr', 'claim_free_years_3yr',
        'cost_avg_5yr', 'cost_max_5yr',
        'unique_conditions_count_lag1', 'had_cardio_lag1', 'had_musculo_lag1',
        'had_gastro_lag1', 'had_chemo_lag1'
    ]

    features = numeric_features + static_features
    features = [f for f in features if f in df_model_data.columns]
    static_features = [f for f in static_features if f in df_model_data.columns]

    print(f"\nTraining with {len(features)} features: {features}")

    # Encode categorical static features (in-place)
    encoders = {}
    for col in static_features:
        le = LabelEncoder()
        df_model_data[col] = df_model_data[col].fillna('MISSING').astype(str)
        df_model_data[col] = le.fit_transform(df_model_data[col])
        encoders[col] = le

    # Ensure all feature columns are numeric
    for f in features:
        if f in df_model_data.columns:
            if df_model_data[f].dtype == 'object':
                df_model_data[f] = pd.to_numeric(df_model_data[f], errors='coerce')
    # final numeric fill
    df_model_data[features] = df_model_data[features].fillna(0)

    # Split: training uses years < validation_year-1, test uses year == validation_year-1
    X_test_df = df_model_data[df_model_data['year'] == validation_year - 1].copy()
    X_train_df = df_model_data[df_model_data['year'] < validation_year - 1].copy()

    X_train = X_train_df[features]
    y_train = X_train_df[target]
    X_test = X_test_df[features]
    y_test = X_test_df[target]

    print(f"Training on {len(X_train)} rows (Years {X_train_df['year'].min()}-{X_train_df['year'].max()})")
    print(f"Validating on {len(X_test)} rows (Predicting year {validation_year})")

    lgb_model = lgb.LGBMRegressor(
        objective='tweedie',
        tweedie_variance_power=1.5,
        metric='rmse',
        n_estimators=1000,
        learning_rate=0.01,
        n_jobs=-1,
        random_state=42
    )

    # Train with early stopping
    lgb_model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(100, verbose=True)],
        categorical_feature=static_features
    )

    y_pred_2025 = lgb_model.predict(X_test)
    y_pred_2025[y_pred_2025 < 0] = 0

    r2 = r2_score(y_test, y_pred_2025)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_2025))

    print("\n" + "=" * 50)
    print(f"--- VALIDATION RESULTS (PREDICTING {validation_year}) ---")
    print(f"R-squared: {r2:.4f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"Actual Avg Cost: {y_test.mean():.2f}")
    print(f"Predicted Avg Cost: {y_pred_2025.mean():.2f}")
    print("=" * 50 + "\n")

    importance_df = pd.DataFrame({
        'feature': lgb_model.feature_name_,
        'importance': lgb_model.feature_importances_
    }).sort_values(by='importance', ascending=False)

    print("\n--- Top 20 Important Features ---")
    print(importance_df.head(20))

    df_val_output = X_test_df.reset_index(drop=True)
    df_val_output['actual_cost'] = y_test.reset_index(drop=True)
    df_val_output['predicted_cost'] = y_pred_2025
    df_val_output.to_csv('validation_predictions_2025.csv', index=False)
    print("Saved validation predictions to: validation_predictions_2025.csv")

    return lgb_model, encoders, features, static_features


# ============================================================================== 
# --- 6. MAIN EXECUTION
# ==============================================================================
def main_train():
    print("--- Starting Model Training ---")

    df_claims, df_members = load_data(CLAIM_FILE_PATH, MEMBER_FILE_PATH)
    if df_claims is None or df_members is None:
        return

    df_master = create_scaffold(df_claims, df_members, START_YEAR, END_YEAR)
    df_model_data, _ = engineer_features(df_master.copy())
    if df_model_data.empty:
        print("Error: No model data created.")
        return

    val_model, encoders, features, static_features = train_and_validate(df_model_data, VALIDATION_YEAR)

    print("\n--- Re-training final model on all data ---")
    target = 'target_cost_next_year'
    X_full = df_model_data[features]
    y_full = df_model_data[target]

    final_model = lgb.LGBMRegressor(**val_model.get_params())
    final_model.set_params(n_estimators=val_model.best_iteration_)
    final_model.fit(X_full, y_full, categorical_feature=static_features)

    joblib.dump(final_model, 'claim_model_method1.lgb')
    joblib.dump(encoders, 'data_encoders.pkl')
    joblib.dump(features, 'feature_list.pkl')

    print("\nArtifacts saved successfully.")
    print("--- Training Complete ---")


if __name__ == "__main__":
    main_train()


In [ ]:
import pandas as pd
import numpy as np
import joblib
import lightgbm as lgb
import warnings

warnings.filterwarnings("ignore")

# ==============================================================================
# --- CONFIGURATION
# ==============================================================================

MODEL_PATH = "trained_claim_model_ukpmi.lgb"
ENCODER_PATH = "trained_encoders_ukpmi.pkl"
FEATURE_LIST_PATH = "trained_features_ukpmi.pkl"

CLAIM_FILE_PATH = "uk_pmi_claims_200k.csv"
MEMBER_FILE_PATH = "uk_pmi_membership_120k.csv"
OUTPUT_FILE = "predicted_claim_costs_2026.csv"

START_YEAR = 2019
PREDICT_YEAR = 2026

# ==============================================================================
# --- FEATURE ENGINEERING (same as training)
# ==============================================================================

def preprocess_for_prediction(df_claims, df_members, start_year, end_year):
    """Builds lag/rolling features up to the given end_year."""
    print(f"Building features up to {end_year}...")

    df_claims["Incurred Date"] = pd.to_datetime(df_claims["Incurred Date"], errors="coerce")
    df_claims["claim_year"] = df_claims["Incurred Date"].dt.year
    df_claims["Claim Amount"] = pd.to_numeric(df_claims["Claim Amount"], errors="coerce").fillna(0)
    df_claims["Calculated Length of Service"] = pd.to_numeric(df_claims["Calculated Length of Service"], errors="coerce").fillna(0)
    df_claims = df_claims.dropna(subset=["Unique Member Reference"])
    df_claims = df_claims[df_claims["claim_year"].between(start_year, end_year)]

    # --- Aggregate claim features ---
    def had_condition(series, keyword):
        return (series.str.contains(keyword, case=False, na=False)).any()

    agg = df_claims.groupby(["Unique Member Reference", "claim_year"]).agg(
        total_claim_amount=("Claim Amount", "sum"),
        total_claim_count=("Claim Amount", "count"),
        total_los=("Calculated Length of Service", "sum"),
        count_inpatient=("Claim Type", lambda x: (x == "Inpatient").sum()),
        unique_conditions_count=("Condition Category", "nunique"),
        had_cardio=("Condition Category", lambda s: had_condition(s, "Cardio")),
        had_musculo=("Condition Category", lambda s: had_condition(s, "Musculo")),
        had_gastro=("Condition Category", lambda s: had_condition(s, "Gastro")),
        had_chemo=("Condition Category", lambda s: had_condition(s, "Chemo")),
    ).reset_index()

    agg = agg.rename(columns={"claim_year": "year"})
    all_members = df_members["Unique Member Reference"].unique()
    all_years = list(range(start_year, end_year + 1))
    scaffold = pd.MultiIndex.from_product([all_members, all_years], names=["Unique Member Reference", "year"])
    df_scaffold = pd.DataFrame(index=scaffold).reset_index()

    df_master = pd.merge(df_scaffold, agg, on=["Unique Member Reference", "year"], how="left").fillna(0)

    # --- Merge membership info ---
    df_master = df_master.merge(df_members, on="Unique Member Reference", how="left")

    # --- Derived features ---
    df_master["age"] = df_master["year"] - pd.to_numeric(df_master["Year of Birth"], errors="coerce")
    df_master["Original Date of Joining"] = pd.to_datetime(df_master["Original Date of Joining"], errors="coerce")
    df_master["tenure"] = df_master["year"] - df_master["Original Date of Joining"].dt.year
    df_master["tenure"] = df_master["tenure"].clip(lower=0)

    df_master = df_master.sort_values(["Unique Member Reference", "year"]).reset_index(drop=True)
    g = df_master.groupby("Unique Member Reference")

    # --- Lag features ---
    df_master["cost_lag1"] = g["total_claim_amount"].shift(1)
    df_master["cost_lag2"] = g["total_claim_amount"].shift(2)
    df_master["claim_count_lag1"] = g["total_claim_count"].shift(1)
    df_master["inpatient_count_lag1"] = g["count_inpatient"].shift(1)
    for c in ["unique_conditions_count", "had_cardio", "had_musculo", "had_gastro", "had_chemo"]:
        df_master[f"{c}_lag1"] = g[c].shift(1)

    # --- Rolling stats ---
    rolling = lambda x, w, f: x.groupby(df_master["Unique Member Reference"]).rolling(w, min_periods=1).agg(f).shift(1).reset_index(level=0, drop=True)
    df_master["cost_avg_3yr"] = rolling(df_master["total_claim_amount"], 3, "mean")
    df_master["cost_max_3yr"] = rolling(df_master["total_claim_amount"], 3, "max")
    df_master["cost_std_3yr"] = rolling(df_master["total_claim_amount"], 3, "std")
    df_master["claim_free_years_3yr"] = rolling((df_master["total_claim_count"] == 0).astype(int), 3, "sum")
    df_master["cost_avg_5yr"] = rolling(df_master["total_claim_amount"], 5, "mean")
    df_master["cost_max_5yr"] = rolling(df_master["total_claim_amount"], 5, "max")

    df_master = df_master.fillna(0)
    return df_master


# ==============================================================================
# --- INFERENCE FUNCTION
# ==============================================================================

def predict_claims_2026():
    print("\n--- Predicting Claim Costs for 2026 ---")

    # Load trained model and preprocessing objects
    model = joblib.load(MODEL_PATH)
    encoders = joblib.load(ENCODER_PATH)
    features = joblib.load(FEATURE_LIST_PATH)

    # Load raw data
    df_claims = pd.read_csv(CLAIM_FILE_PATH)
    df_members = pd.read_csv(MEMBER_FILE_PATH)

    # Build all features up to 2025
    df_master = preprocess_for_prediction(df_claims, df_members, START_YEAR, 2025)
    df_latest = df_master[df_master["year"] == 2025].copy()

    # Encode categoricals
    for col, le in encoders.items():
        if col in df_latest.columns:
            df_latest[col] = df_latest[col].astype(str)
            unseen = ~df_latest[col].isin(le.classes_)
            if unseen.any():
                df_latest.loc[unseen, col] = "MISSING"
                le.classes_ = np.append(le.classes_, "MISSING")
            df_latest[col] = le.transform(df_latest[col])

    # Align features
    for f in features:
        if f not in df_latest.columns:
            df_latest[f] = 0
    df_latest = df_latest[features].fillna(0)

    # Predict
    preds = model.predict(df_latest)
    preds = np.maximum(preds, 0)

    df_output = pd.DataFrame({
        "Unique Member Reference": df_master[df_master["year"] == 2025]["Unique Member Reference"],
        "predicted_claim_cost_2026": preds
    })

    df_output.to_csv(OUTPUT_FILE, index=False)
    print(f"✅ Predictions for 2026 saved to {OUTPUT_FILE}")
    print(df_output.head())


if __name__ == "__main__":
    predict_claims_2026()
